<a href="https://colab.research.google.com/github/Rian021102/Claim-Checking-Using-Bert/blob/main/ClaimBuster_BERT_Rian_Newdataset_ver02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Loading Data

In [ ]:
!pip install torch
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:


%tensorflow_version 2.x
import os
from shutil import rmtree
import math
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


Define parameters for text vectorization

In [ ]:
max_len = 500
top_words = 5000
max_words = 10000
path_to_data = './'
glove_dir = './'
embedding_dim = 300
embedding_file_name = 'glove.6B.300d.txt'

Obtain and process data

# Train on Full Dataset Train.json and Test.json

In [ ]:
df_train=pd.read_csv('ClaimBuster_2c_modified_from_3c.csv')

In [ ]:
df_train

,Unnamed: 0,text,label
0,0,"Rayburn says, yeah, and that guy never ran for...",1
1,1,That's another reason to get the deficit down.,0
2,2,"That is, I want to bring down rates.",0
3,3,It would borrow up to $3 trillion.,1
4,4,Our country has sold out to foreign lobbyists.,1
...,...,...,...
23528,23528,"Thirdly, we have to bring interest rates down,...",0
23529,23529,If the president had shown the patience to go ...,0
23530,23530,I want to focus on results.,0
23531,23531,I said that we would create jobs for our peopl...,1


In [ ]:
!wget -O claimbuster_test.json https://raw.githubusercontent.com/utaresearch/claimbuster-spotter/master/data/two_class/test.json


--2023-03-22 11:03:01--  https://raw.githubusercontent.com/utaresearch/claimbuster-spotter/master/data/two_class/test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121150 (118K) [text/plain]
Saving to: ‘claimbuster_test.json’

claimbuster_test.js 100%[===================>] 118.31K  --.-KB/s    in 0.002s  

2023-03-22 11:03:02 (47.4 MB/s) - ‘claimbuster_test.json’ saved [121150/121150]



In [ ]:
test_loc = path_to_data + 'claimbuster_test.json'
df_test = pd.read_json(test_loc)

In [ ]:
df_test

,sentence_id,label,text
0,18737,1,We worked very closely with the foreign minist...
1,32170,1,"Right now, we are at 90 percent health insuran..."
2,29250,1,Donald thinks that climate change is a hoax pe...
3,8208,1,We find that the prices you pay went up five t...
4,34264,1,"And this is what's caused the great migration,..."
...,...,...,...
697,25611,0,"When you make that judgment, you are deciding ..."
698,23590,0,So you get a pretty good sense of where he's h...
699,25055,0,That is a very important objective -- to make ...
700,26524,0,Now that's not a government takeover.


In [ ]:
X=list(df_train['text'])
y=list(df_train['label'])

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the pretrained tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the pretrained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Set the device (CPU or GPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define the input text
text = "This is some example text."

# Tokenize the text
input_ids = tokenizer.encode(text, return_tensors='pt').to(device)

# Make a prediction
with torch.no_grad():
    outputs = model(input_ids)
    predictions = torch.argmax(outputs[0], dim=1)

# Print the predictions
print(predictions)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

tensor([1], device='cuda:0')


In [ ]:
train_encodings = tokenizer(X_train, padding=True, truncation=True)
val_encodings = tokenizer(X_val, padding=True, truncation=True)


In [ ]:
import torch

class ClaimbusterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = ClaimbusterDataset(train_encodings, y_train)
val_dataset = ClaimbusterDataset(val_encodings, y_val)


In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
print (train_dataset)

In [ ]:
# Define the training arguments
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # learning rate
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=500,               # log after every 500 steps
    evaluation_strategy='steps',     # evaluate after every epoch
    save_total_limit=2,              # number of checkpoints to save
    load_best_model_at_end=True,     # load the best model when training ends
    metric_for_best_model='f1'       # use F1 score to determine the best model
)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
    }


In [ ]:
# Initialize the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.446200,0.361810,0.847461,0.850125
1000,0.318100,0.300722,0.871468,0.871104
1500,0.260200,0.334498,0.873168,0.874025
2000,0.211000,0.383672,0.873805,0.871249
2500,0.184700,0.447137,0.874867,0.874669
3000,0.111200,0.525791,0.874017,0.874318
3500,0.119900,0.490489,0.874867,0.875023


TrainOutput(global_step=3531, training_loss=0.23484561545286392, metrics={'train_runtime': 2182.0024, 'train_samples_per_second': 25.884, 'train_steps_per_second': 1.618, 'total_flos': 5427377766649080.0, 'train_loss': 0.23484561545286392, 'epoch': 3.0})

In [ ]:
# Evaluate the model on the validation dataset
eval_results = trainer.evaluate(val_dataset)

print(eval_results)

{'eval_loss': 0.4904889464378357, 'eval_accuracy': 0.874867219035479, 'eval_f1': 0.8750228365947503, 'eval_runtime': 41.8862, 'eval_samples_per_second': 112.376, 'eval_steps_per_second': 7.043, 'epoch': 3.0}


In [ ]:
import os
output_dir='modelsave'
model_to_save=model.module if hasattr(model,'module') else model
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('modelsave/tokenizer_config.json',
 'modelsave/special_tokens_map.json',
 'modelsave/vocab.txt',
 'modelsave/added_tokens.json')

In [ ]:
model_dir = 'modelsave/'
model = model=BertForSequenceClassification.from_pretrained(f"{model_dir}")
tokenizer = BertTokenizer.from_pretrained(f"{model_dir}")


In [ ]:
inputs = tokenizer(list(df_test['text']), padding=True, truncation=True, return_tensors="pt")


In [ ]:
with torch.no_grad():
    logits = model(**inputs).logits


In [ ]:
y_predict = torch.argmax(logits, dim=1).tolist()


In [ ]:
from sklearn.metrics import accuracy_score

y_true = list(df_test['label'])
accuracy = accuracy_score(y_true, y_predict)


In [ ]:
print(accuracy)

0.9230769230769231


In [ ]:
from sklearn.metrics import classification_report

y_true = list(df_test['label'])
target_names = ['class 0', 'class 1']  # Replace with your actual class names
report = classification_report(y_true, y_predict, target_names=target_names)

print(report)


              precision    recall  f1-score   support

     class 0       0.97      0.87      0.92       351
     class 1       0.89      0.97      0.93       351

    accuracy                           0.92       702
   macro avg       0.93      0.92      0.92       702
weighted avg       0.93      0.92      0.92       702

